# Homework 5: Due 16 Oct 2017 23h59 (Mon; midnight)

You may do this homework in groups of up to three

This homework is very much like an experiment. Here's what we're looking for in the submission report:

Page 1: Table listing the experiments carried out with the following columns.

 Size of the fixed length sample    Overlap (0-X%)    K- value    Classifier    Accuracy
We expect you to have tried at least 2 values of K and at least 2 different lengths of the windows for quantization.
NOTE: For K-means please also list if you used standard K-means or hierarchical
NOTE: For classification, please mention what your test-train split is and how you created the test-train split.
  
Page 2:
1. Histograms of the mean quantized vector (Histogram of cluster centres like in the book) for each activity with the K value that gives you the highest accuracy. (Please state the K value)
2. Class confusion matrix from the classifier that you used. Please make sure to label the row/columns of the matrix so that we know which row corresponds to what.
   
Page 3: Code snippets (few lines) highlighting the following:
  1. segmentation of the vector
  2. k-means
  3. generating the histogram
  4. classification
  
2. Page 4 and beyond:
  1. Any relevant code screenshot
  
We will also be awarding extra points for in-depth experimentation and analysis.
   
What we're mostly looking for is if how likely are we to be able to replicate the results following the steps you took. So, things you should look for are patterns in the confusion matrix between activities which are similar. Explaining your results in a few lines would thus be very useful. These page restrictions are not _STRICT_ in the sense that you can have an experiments table that flows into two pages if you did a lot of experiments, in such cases please make sure to mark both the pages for that question while uploading.
    
Really sorry again for the long delay in posting the instructions. If there are any issues that this causes please let us know and we will try to alleviate them.

## Problem 1
You may use any programming language that amuses you for this homework.

Do problem 9.4 in the 18-Sep version of the textbook.

## Import functions, dataset

In [1]:
%reload_ext autoreload
%autoreload 2
from hw5_funct import *
X_data, Y_data, activities = get_data('./data/')

## Test train split

In [2]:
from sklearn.model_selection import train_test_split

# Remember to stratify by labels.
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, stratify=Y_data)

## Part (a)

Build a classifier that classifies sequences into one of the 14 activities provided.
To make features, you should vector quantize, then use a histogram
of cluster centers (as described in the subsection; this gives a pretty explicit
set of steps to follow). You will find it helpful to use hierarchical
k-means to vector quantize. You may use whatever multi-class classifier
you wish, though I’d start with R’s decision forest, because it’s easy to
use and effective. You should report (a) the total error rate and (b) the
class confusion matrix of your classifier.

## Vector Quantization

In [13]:
# Adjust window and K here. Window should be some multiple of 3.
window = 3*9
K = 400

# Get VQ dictionary from training dataset.
vq_dict = get_vq_dict(X_train, window, K)

# Apply VQ to training and test dataset.
X_train_VQ = vec_quantize_signals(X_train, vq_dict)
X_test_VQ = vec_quantize_signals(X_test, vq_dict)

## TODO: Plot per-class histograms.

In [15]:
import matplotlib.pyplot as plt

vqs = []
for i in range(len(activities)):
    vqs.append([])

for i in range(len(X_train_VQ)):
    label = Y_train[i]
    vqs[label].append(X_train_VQ[i])
        
for i in range(len(activities)):
    plot = np.mean(vqs[i], axis = 0)
    plt.figure()
    plt.ylim([0, 0.2])
    plt.bar(range(K), plot)
    title = activities[i]
    plt.title(title)
#     plt.show()
    filename = "./hist/%s.png" %(title)
    plt.savefig(filename)
    plt.close('all')


## Classification

## Classify with Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=100, max_depth=50)
rf.fit(X_train_VQ, Y_train)

# Print accuracy.
print(rf.score(X_test_VQ,Y_test))

# Print class confusion matrix.
Y_pred = rf.predict(X_test_VQ)
print(confusion_matrix(Y_test,Y_pred))

## Classify with kNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_VQ, Y_train)

# Print accuracy.
print(knn.score(X_test_VQ,Y_test))

# Print class confusion matrix.
Y_pred = knn.predict(X_test_VQ)
print(confusion_matrix(Y_test,Y_pred).shape)

## Part (b)

Now see if you can improve your classifier by (a) modifying the number
of cluster centers in your hierarchical k-means and (b) modifying the size
of the fixed length samples that you use.

## Vector Quantization with overlapping window

In [7]:
# Adjust window and K here. Window should be some multiple of 3.
window = 3*9
K = 40
overlap = 0.3

# Get VQ dictionary from training dataset.
vq_dict = get_vq_dict_overlap(X_train, window, K, overlap)

# Apply VQ to training and test dataset.
X_train_VQ = vec_quantize_signals(X_train, vq_dict)
X_test_VQ = vec_quantize_signals(X_test, vq_dict)

## Classification

## Classify with Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=100, max_depth=50)
rf.fit(X_train_VQ, Y_train)

# Print accuracy.
print(rf.score(X_test_VQ,Y_test))

# Print class confusion matrix.
Y_pred = rf.predict(X_test_VQ)
print(confusion_matrix(Y_test,Y_pred))

0.7261904761904762
[[ 1  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0 12  0  2  0  0  0  0  0  0  0  1  0  5]
 [ 0  0  3  0  1  0  0  1  0  1  0  0  0  0]
 [ 0  2  0  4  0  0  0  0  0  0  0  1  0  1]
 [ 0  0  0  0 17  0  0  1  0  2  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 16  0  0  3  1  0  0]
 [ 0  0  0  0  0  0  0  1  1  0  4  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 20  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0 16  3  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  4 16  0  0]
 [ 0  0  0  0  3  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  1  0  0  0  2  0  0  0  0  0 15]]


## Vector Quantization with hierachical k-means

In [29]:
# Adjust window and K here. Window should be some multiple of 3.
window = 3*32
K_0 = 40
K_1 = 10
sample = 50

# Get VQ dictionary from training dataset.
vq_dict = get_vq_dict_hierarchical(X_train, window, K_0, K_1, sample)

# Apply VQ to training and test dataset.
X_train_VQ = vec_quantize_signals(X_train, vq_dict)
X_test_VQ = vec_quantize_signals(X_test, vq_dict)

## TODO: Plot per-class histograms.

In [30]:
import matplotlib.pyplot as plt

vqs = []
for i in range(len(activities)):
    vqs.append([])

for i in range(len(X_train_VQ)):
    label = Y_train[i]
    vqs[label].append(X_train_VQ[i])
        
for i in range(len(activities)):
    plot = np.mean(vqs[i], axis = 0)
    plt.figure()
    plt.ylim([0, 0.2])
    plt.bar(range(len(plot)), plot)
    title = activities[i]
    plt.title(title)
#     plt.show()
    filename = "./hist/%s.png" %(title)
    plt.savefig(filename)
    plt.close('all')


## Classification

## Classify with Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=100, max_depth=50)
rf.fit(X_train_VQ, Y_train)

# Print accuracy.
print(rf.score(X_test_VQ,Y_test))

# Print class confusion matrix.
Y_pred = rf.predict(X_test_VQ)
print(confusion_matrix(Y_test,Y_pred))
print(activities)

0.7857142857142857
[[ 2  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 16  0  0  0  0  0  0  0  0  0  0  0  4]
 [ 0  0  6  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  7  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 18  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0 13  0  0  1  6  0  0]
 [ 0  0  0  0  0  0  0  2  0  1  3  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 20  0  0  0  0]
 [ 0  0  0  0  1  0  0  3  0  2 10  4  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  1 19  0  0]
 [ 0  0  0  0  2  0  0  0  0  0  0  0  1  0]
 [ 0  1  0  0  0  0  0  0  0  0  0  0  0 19]]
['Brush_teeth', 'Climb_stairs', 'Comb_hair', 'Descend_stairs', 'Drink_glass', 'Eat_meat', 'Eat_soup', 'Getup_bed', 'Liedown_bed', 'Pour_water', 'Sitdown_chair', 'Standup_chair', 'Use_telephone', 'Walk']
